In [11]:
import os
import tqdm
import pickle
import rasterio
import argparse
import time
import shutil
import sys
import warnings
from rasterio.windows import Window
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from joblib import Parallel, delayed
from scipy.ndimage import gaussian_filter
from scipy.stats import norm as scipy_norm

sys.path.append('../')
from utils.model_utils import fit_detection_model, fit_migration_model, m_numpy
from utils.spatial_utils import DistributionMap, DataMap, fit_GWR, binary_search_dec, binary_search_inc
from utils.eval_utils import fast_auc
start_time = time.time()

# %% Paths and constants
# Detection param prior mean and penalty
beta_prec = 1/100.0
beta_mean =  np.array([-2.0, -2.0, 1.0, 0.0, 0.0])
# Migration functional penalty strength
theta_prec = 1/100.0
# maximum neighborhood size and Gaussian kernel radius
r_nh = 5.0
r_kernel = 2.5

path_project = "/scratch/project_2003104/gtikhono/realtime_birds"
dir_orig_data = "orig_data"
dir_data = "data"
dir_results = "results"
df_sp_model = pd.read_csv(os.path.join(path_project, dir_data, "modeled_species.csv"))
sp_list = list(df_sp_model.species)



sp = sp_list[19]
print(sp, flush=True)
detection_train_range = [1, 365]
migration_train_range = [1, 365]
spatial_train_range = [1, 365]
test_range = [366, 730]
prior_type = "transect"
save_new_prior = bool(1)
name_new_prior = "appTest"
save_prediction = bool(1)
save_images = bool(1)
reset_prior_detection = bool(1)
reset_prior_migration = bool(0)
reset_prior_spatial = bool(0)
factor = 10
jn = 4


path_sp = os.path.join(path_project, dir_data, "species", sp)
path_result = os.path.join(path_project, dir_results, sp)
os.makedirs(path_result, exist_ok=True)
suffix_args = [prior_type] + detection_train_range + migration_train_range + spatial_train_range + \
    [reset_prior_detection,reset_prior_migration,reset_prior_spatial] + test_range
suffix_result = "%s_det(%d_%d)_mig(%d_%d)_dyn(%d_%d)_test%d%d%d(%d_%d)" % tuple(suffix_args)

if prior_type == "transect":
    path_a = os.path.join(path_sp, sp + "_a.tif")
    path_va = os.path.join(path_sp, sp + "_va.tif")
else:
    path_a = os.path.join(path_sp, sp + f"_a_{prior_type}.tif")
    path_va = os.path.join(path_sp, sp + f"_va_{prior_type}.tif")

# %% Loading non-raster data
with open(os.path.join(path_project, dir_data, "XData.pickle"), 'rb') as handle:
    XData = pickle.load(handle)
short = (XData["rec_class"] == "short").to_numpy().astype(int)
long = (XData["rec_class"] == "long").to_numpy().astype(int)
point = ((XData["rec_class"]!="short")&(XData["rec_class"]!="long")).to_numpy().astype(int)
log_duration = XData["log_duration"].to_numpy()
lats = XData["lat"].to_numpy()
lons = XData["lon"].to_numpy()
days = XData["j.date"].to_numpy() # in {1,...,730}, but later migration model expects values in {1,...,365}. Use days%365!
ones = np.ones(XData.shape[0])

with open(os.path.join(path_project, dir_data, "migration_prior_params.pickle"), 'rb') as handle:
    prior_m_params = pickle.load(handle)
prior_m_params_u_days = prior_m_params.loc[sp][-2:].to_numpy()
prior_m_params = prior_m_params.loc[sp][:6].to_numpy()

with open(os.path.join(path_sp, sp+"_prior.pickle"), 'rb') as handle:
    species = pickle.load(handle)
species["prior.s.L"] = pd.Series(scipy_norm.ppf(species["prior.s"]), index=species["prior.s"].index)
if species["prior.s.L"].isna().any() or species["prior.s.L"].isin([np.inf, -np.inf]).any():
    print("NaNs or infs introduced when mapping s -> Phi^{-1}(s)")
y = species["y"].to_numpy()
prior_s = species["prior.s"].to_numpy()
prior_d_a_transect = species["prior.d.a"].to_numpy()
prior_d_b_transect = species["prior.d.b"].to_numpy()
with np.errstate(invalid='ignore'):
    prior_d_transect = np.minimum(prior_d_a_transect + prior_d_b_transect, 1)
prior_m = species["prior.m"].to_numpy()
prior_sL = species["prior.s.L"].to_numpy()
complete = (species["complete"] & ~(species["prior.s.L"].isna() | species["prior.s.L"].isin([np.inf, -np.inf]))).to_numpy()

detection_train_idx = complete*(detection_train_range[0] <= days)*(days <= detection_train_range[1])
migration_train_idx = complete*(migration_train_range[0] <= days)*(days <= migration_train_range[1])
spatial_train_idx = complete*(spatial_train_range[0] <= days)*(days <= spatial_train_range[1])
test_idx = complete*(test_range[0] <= days)*(days <= test_range[1])

026_anthus_trivialis
NaNs or infs introduced when mapping s -> Phi^{-1}(s)


In [12]:
# %% Prior predictions
prior_preds = prior_m*prior_s*prior_d_transect
prior_AUC = fast_auc(y[test_idx], prior_preds[test_idx])
prior_R2 = prior_preds[(y==1)*test_idx].mean() - prior_preds[(y==0)*test_idx].mean()
print("Prior AUC:", np.round(prior_AUC,3), flush=True)
print("Prior R2:", np.round(prior_R2,3), flush=True)
# with open(os.path.join(path_result, "%s_predict_prior_%s.txt" % (sp, suffix_result)), "w") as f:
#   f.write("AUC %f\nR2 %f" % (prior_AUC, prior_R2))
  
# %% Reading prior spatial predictions
with rasterio.open(path_a) as src:
    height_ha, width_ha = src.height, src.width
    transform_ha = src.transform
    height_km = height_ha//factor
    width_km = width_ha//factor
    window = Window(0, 0, width_km*factor, height_km*factor)
    a_ha = src.read(1, window=window)
    height_ha, width_ha = a_ha.shape
    transform_ha = src.window_transform(window) 
    profile = src.profile
    profile.update({"height": height_ha, "width": width_ha,
                    "transform": transform_ha, "nodata": np.nan})

with rasterio.open(path_va) as src:
	va_ha = src.read(1, window=window)
va_ha[(np.isnan(va_ha))&(~np.isnan(a_ha))] = 1.0 # ensure a_map != nan implies va_map != nan
va_ha[np.isnan(a_ha)] = np.nan
va_ha = np.clip(va_ha, 1e-4, 1) # clip very small variances to avoid numerical issues with precision

# %% Calculating and downscaling grids 
a_km = a_ha.copy()
a_km = a_km.reshape(height_km, factor, width_km, factor)
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "Mean of empty slice")
    a_km = np.nanmean(a_km, axis=(1, 3))
va_km = va_ha.copy()
va_km = va_km.reshape(height_km, factor, width_km, factor)
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "Mean of empty slice")
    va_km = np.nanmean(va_km, axis=(1, 3))
transform_km = transform_ha*transform_ha.scale(factor, factor)

# extract grids
ys_km = np.arange(height_km)
_, lat_grid_km = rasterio.transform.xy(transform_km, ys_km, np.zeros_like(ys_km))
lat_grid_km = np.array(lat_grid_km)
xs_km = np.arange(width_km)
lon_grid_km, _ = rasterio.transform.xy(transform_km, np.zeros_like(xs_km), xs_km)
lon_grid_km = np.array(lon_grid_km)

ys_ha = np.arange(height_ha)
_, lat_grid_ha = rasterio.transform.xy(transform_ha, ys_ha, np.zeros_like(ys_ha))
lat_grid_ha = np.array(lat_grid_ha)
xs_ha = np.arange(width_ha)
lon_grid_ha, _ = rasterio.transform.xy(transform_ha, np.zeros_like(xs_ha), xs_ha)
lon_grid_ha = np.array(lon_grid_ha)

lat_min, lat_max = max(lat_grid_ha[-1], lat_grid_km[-1]), min(lat_grid_ha[0], lat_grid_km[0]) 
lon_min, lon_max = max(lon_grid_ha[0], lon_grid_km[0]), min(lon_grid_ha[-1], lon_grid_km[-1])
lons_clipped = np.clip(lons, lon_min, lon_max)
lats_clipped = np.clip(lats, lat_min, lat_max)

cell_idx = np.arange(height_km*width_km).reshape(height_km, width_km)
rows_to_grid = rasterio.transform.rowcol(transform_km, lons_clipped, lats_clipped)
rows_to_idx = cell_idx[rows_to_grid]

cell_idx_ha = np.arange(height_ha*width_ha).reshape(height_ha, width_ha)
rows_to_grid_ha = rasterio.transform.rowcol(transform_ha, lons_clipped, lats_clipped)
rows_to_idx_ha = cell_idx_ha[rows_to_grid_ha]

prior_d_a = a_ha.flatten()[rows_to_idx_ha]
with np.errstate(invalid='ignore'):
    prior_d = np.minimum(prior_d_a + prior_d_b_transect, 1) if prior_type == "transect" else prior_d_a



Prior AUC: 0.776
Prior R2: 0.152


In [13]:
# %% Training detection model and making prediction
tau_detection = prior_m*prior_d
X_detection = np.c_[ones*short, ones*long, prior_sL, short*log_duration, long*log_duration, point]
beta = fit_detection_model(y[detection_train_idx], X_detection[detection_train_idx,:5], 
                           tau_detection[detection_train_idx,], beta_mean, beta_prec)
point_intercept = (beta[0] + beta[1] + (beta[3] + beta[4])*np.log(300))/2 # irrelevant for 2023 but needed for 2024
beta = np.append(beta, point_intercept)
post_s = scipy_norm.cdf(X_detection@beta)

post_detection = post_s*tau_detection
detection_AUC = fast_auc(y[test_idx], post_detection[test_idx])
detection_R2 = post_detection[(y==1)*test_idx].mean() - post_detection[(y==0)*test_idx].mean()
print("AUC after updating detection:", np.round(detection_AUC,3), flush=True)
print("R2 after updating detection:", np.round(detection_R2,3), flush=True)
# with open(os.path.join(path_result, "%s_predict_detection_%s.txt" % (sp, suffix_result)), "w") as f:
#   f.write("AUC %f\nR2 %f" % (detection_AUC, detection_R2))

Training detection model:   0%|          | 0/10000 [00:00<?, ?it/s]

tensor([-19.9051, -19.6972, -19.6978,  ..., -30.1613, -30.1584, -30.1579],
       dtype=torch.float64, grad_fn=<MvBackward0>)
tensor(-0.2184, dtype=torch.float64, grad_fn=<MeanBackward0>) tensor(-0.0100, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor([-0.0100, -0.0100,  0.0000,  0.0000,  0.0000], dtype=torch.float64)
tensor([-3.0000, -3.0000,  1.0000,  0.0000,  0.0000], dtype=torch.float64,
       requires_grad=True)
tensor([-19.9051, -19.6971, -19.6978,  ..., -30.1613, -30.1583, -30.1579],
       dtype=torch.float64, grad_fn=<MvBackward0>)
tensor(-0.2184, dtype=torch.float64, grad_fn=<MeanBackward0>) tensor(-0.0100, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor([-0.0100, -0.0100,  0.0000,  0.0000,  0.0000], dtype=torch.float64)
tensor([-3.0000, -3.0000,  1.0000,  0.0000,  0.0000], dtype=torch.float64,
       requires_grad=True)
tensor([-19.9051, -19.6971, -19.6978,  ..., -30.1613, -30.1583, -30.1579],
       dtype=torch.float64, grad_fn=<MvBackward0>)
tensor(-0.2184, dtype=

Training detection model:   3%|▎         | 251/10000 [00:10<06:29, 25.00it/s]

tensor(-0.2184, dtype=torch.float64, grad_fn=<MeanBackward0>) tensor(-0.0100, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor([-0.0100, -0.0100,  0.0000,  0.0000,  0.0000], dtype=torch.float64)
tensor([-2.9975, -2.9975,  1.0000,  0.0000,  0.0000], dtype=torch.float64,
       requires_grad=True)
tensor([-19.9026, -19.6947, -19.6954,  ..., -30.1589, -30.1559, -30.1554],
       dtype=torch.float64, grad_fn=<MvBackward0>)
tensor(-0.2184, dtype=torch.float64, grad_fn=<MeanBackward0>) tensor(-0.0100, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor([-0.0100, -0.0100,  0.0000,  0.0000,  0.0000], dtype=torch.float64)
tensor([-2.9975, -2.9975,  1.0000,  0.0000,  0.0000], dtype=torch.float64,
       requires_grad=True)
tensor([-19.9026, -19.6947, -19.6953,  ..., -30.1589, -30.1559, -30.1554],
       dtype=torch.float64, grad_fn=<MvBackward0>)
tensor(-0.2184, dtype=torch.float64, grad_fn=<MeanBackward0>) tensor(-0.0100, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor([-0.0100, -0.0100,

Training detection model:   3%|▎         | 288/10000 [00:11<06:28, 25.02it/s]

tensor(-0.2184, dtype=torch.float64, grad_fn=<MeanBackward0>) tensor(-0.0099, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor([-0.0100, -0.0100,  0.0000,  0.0000,  0.0000], dtype=torch.float64)
tensor([-2.9971, -2.9971,  1.0000,  0.0000,  0.0000], dtype=torch.float64,
       requires_grad=True)
tensor([-19.9022, -19.6943, -19.6950,  ..., -30.1585, -30.1555, -30.1551],
       dtype=torch.float64, grad_fn=<MvBackward0>)
tensor(-0.2184, dtype=torch.float64, grad_fn=<MeanBackward0>) tensor(-0.0099, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor([-0.0100, -0.0100,  0.0000,  0.0000,  0.0000], dtype=torch.float64)
tensor([-2.9971, -2.9971,  1.0000,  0.0000,  0.0000], dtype=torch.float64,
       requires_grad=True)
tensor([-19.9022, -19.6943, -19.6950,  ..., -30.1585, -30.1555, -30.1551],
       dtype=torch.float64, grad_fn=<MvBackward0>)
tensor(-0.2184, dtype=torch.float64, grad_fn=<MeanBackward0>) tensor(-0.0099, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor([-0.0100, -0.0100,

KeyboardInterrupt: 

In [ ]:
tau_detection[detection_train_idx,].sum()

In [4]:
beta_mean
beta_prec

0.01

In [8]:
species["prior.s"].min()

2.565286078682324e-215

In [9]:
import torch

(torch.tensor(X_detection[detection_train_idx,:5], dtype=torch.float32)@torch.tensor([-3.0, -3.0, 1.0, 0.0, 0.0]))

tensor([-19.9051, -19.6972, -19.6978,  ..., -30.1613, -30.1584, -30.1579])